# ML Hands-on Challenge - Getting started 

The goal of this notebook is to explore the data that we have provided from the ML hands-on challenge. You will learn more about the CATH labels, how to visualize the protein structure, and challenges you will have to handle (e.g. gaps in structure). 

## 0. Setup

In [ ]:
!pip install biopython

In [ ]:
!pip install py3dmol

In [4]:
import pandas as pd
import numpy as np
import sys
import glob
import os
import Bio.PDB.PDBParser
import py3Dmol
import warnings

warnings.filterwarnings("ignore", message="Used element '.' for Atom")

architecture_names = {(1, 10): "Mainly Alpha: Orthogonal Bundle",
                      (1, 20): "Mainly Alpha: Up-down Bundle",
                      (2, 30): "Mainly Beta: Roll",
                      (2, 40): "Mainly Beta: Beta Barrel",
                      (2, 60): "Mainly Beta: Sandwich",
                      (3, 10): "Alpha Beta: Roll",
                      (3, 20): "Alpha Beta: Alpha-Beta Barrel",
                      (3, 30): "Alpha Beta: 2-Layer Sandwich",
                      (3, 40): "Alpha Beta: 3-Layer(aba) Sandwich",
                      (3, 90): "Alpha Beta: Alpha-Beta Complex"}

## 1. Opening the data

In [5]:
# Open the training data sequences and structure
data = pd.read_csv('cath_w_seqs_share.csv', index_col=0)
data

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices
0,2w3sB01,3,90,1170,50,2.60,2w3s,SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...,"[(2, 124)]"
1,3be3A00,2,30,30,320,2.04,3be3,QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...,"[(6, 81)]"
2,3zq4C03,3,10,20,580,3.00,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]"
3,1peqA03,1,10,1650,20,2.80,1peq,DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...,"[(294, 346)]"
4,1bdoA00,2,40,50,100,1.80,1bdo,EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...,"[(77, 156)]"
...,...,...,...,...,...,...,...,...,...
6268,2yyiA02,2,40,110,10,1.66,2yyi,ATTHALTNPQVNRARPPSGQPDPYIPVGVVKQTEKGIVVRGARMTA...,"[(139, 266)]"
6269,4mo0A00,3,30,780,10,2.10,4mo0,EQKIKIYVTKRRFGKLMTIIEGFDTSVIDLKELAKKLKDICACGGT...,"[(24, 102)]"
6270,1vq8X00,3,10,440,10,2.20,1vq8,ERVVTIPLRDARAEPNHKRADKAMILIREHLAKHFSVDEDAVRLDP...,"[(7, 88)]"
6271,1ze3D00,3,10,20,410,1.84,1ze3,DLYFNPRFLLSRFENGQELPPGTYRVDIYLNNGYMATRDVTFNTGD...,"[(1, 125)]"


In [6]:
# The CATH classification for a protein can be determined by concatenating the columns
example_cath_id = data['cath_id'][0]
example_class = data['class'][0]
example_arch = f"({example_class},{data['architecture'][0]})"
example_topo = f"({example_class},{data['architecture'][0]},{data['topology'][0]})"
example_superfam = f"({example_class},{data['architecture'][0]},{data['topology'][0]},{data['superfamily'][0]})"

print(f"""
Protein domain with cath id {example_cath_id} is in class {example_class}, \
architecture {example_arch}, topology {example_topo}, and superfamily {example_superfam}.
""")


Protein domain with cath id 2w3sB01 is in class 3, architecture (3,90), topology (3,90,1170), and superfamily (3,90,1170,50).



For this example cath id "2w3sB01," your model should ideally classify it as architecture (3,90). Also, from this example, you can deduce how to construct the labels that you need.

In [ ]:
!unzip pdb_share.zip

In [30]:
# The sequences come from the PDB files
from Bio.PDB.Polypeptide import protein_letters_3to1

def get_sequence_from_pdb(pdb_filename):
    pdb_parser = Bio.PDB.PDBParser()
    structure = pdb_parser.get_structure(pdb_filename, pdb_filename)
    assert len(structure) == 1

    seq = []

    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_id()[0] == " ":  # This checks if it's a standard residue
                    seq.append(protein_letters_3to1[residue.get_resname()])
                else:
                    print('nonstandard', residue.get_id())

    return ''.join(seq)

example_seq = get_sequence_from_pdb(f"pdb_share/{example_cath_id}")
print(f"The sequence for cath id {example_cath_id} is {example_seq}")

# Check that it matches the data file
data['sequences'][0] == example_seq

The sequence for cath id 2w3sB01 is SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAITGLDLEPVRESPGVIAVFTAADLPHDNDASPAPSPEPVLATGEVHFVGQPIFLVAATSHRAARIAARKARITYAPRP


True

## 2. Visualize some examples

We are grouping at the CATH architecture level as this is the level that your models will be classifying the protein domains.

There are 10 CATH architectures that the protein domains can be classified into, and we will explore examples from each architecture.

In [7]:
# Load sequence and structure for one example for each architecture
cath_examples = data.groupby(['class', 'architecture'])[['cath_id','sequences']].first()
cath_examples

cath_id                                          sequences
class architecture                                                            
1     10            1peqA03  DITFRLAKENAQMALFSPYDIQRRYGKPFGDIAISERYDELIADPH...
      20            4w8pA02  GQQGLVEPTQFARANQAIQMACQSLGEPGCTQAQVLSAATIVAKHT...
2     30            3be3A00  QDFRPGVYRHYKGDHYLALGLARADETDEVVVVYTRLYARAGLPST...
      40            1bdoA00  EISGHIVRSPMVGTFYRTPSPDAKAFIEVGQKVNVGDTLCIVEAMK...
      60            4fgmA01  IAYDITPIDLHGHLFNVSLTIEQTNDEQELWLPNWIPGSYLIRDFS...
3     10            3zq4C03  DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...
      20            1o1zA00  HHHHVIVLGHRGYSAKYLENTLEAFMKAIEAGANGVELDVRLSKDG...
      30            1r7lA00  LYFQSNAMKPRDINKLIASKIFGYEIKDDNIIKDGRYRLGIPLYSQ...
      40            3i9v600  EREGILFTTLEKLVAWGRSNSLWPATFGLACCAIEMMASTDARQAD...
      90            2w3sB01  SVGKPLPHDSARAHVTGQARYLDDLPCPANTLHLAFGLSTEASAAI...

In [88]:
def view_structure(pdb_filename, name, gaps=[], width=300, height=300):
  pdb_parser = Bio.PDB.PDBParser()
  structure = pdb_parser.get_structure(pdb_filename, pdb_filename)

  # Add the model and set the cartoon style
  viewer = py3Dmol.view(query=f'arch: {name}, pdb: {pdb_filename}', width=width, height=height)
  viewer.addModel(open(pdb_filename, 'r').read(), 'pdb')
  viewer.setStyle({'cartoon': {'color': 'spectrum'}})

  if gaps:
    # Add dashed lines for gaps
    for chain_id, start_res, end_res in gaps:
        try:
            start_residue = structure[0][chain_id][start_res-1]
            end_residue = structure[0][chain_id][end_res]

            # Get coordinates and convert to Python float
            start_coords = [float(coord) for coord in start_residue['CA'].get_coord()]
            end_coords = [float(coord) for coord in end_residue['CA'].get_coord()]

            # Add dashed cylinders for missing residues
            viewer.addCylinder({'start': {'x': start_coords[0], 'y': start_coords[1], 'z': start_coords[2]},
                                'end': {'x': end_coords[0], 'y': end_coords[1], 'z': end_coords[2]},
                                'radius': 0.1, 'color': 'red', 'dashed': True, 'fromCap': 1, 'toCap': 1})
        except KeyError:
          print(f"Residue {start_res} or {end_res} in chain {chain_id} not found.")

  viewer.zoomTo()
  return viewer

In [21]:
import py3Dmol
from IPython.display import display, HTML

pdb_dir = 'pdb_share'
num_columns = [2, 3, 5]  # Number of columns in the grid
# titles = ['Structure 1', 'Structure 2', 'Structure 3', 'Structure 4']

# Creating HTML table for the grid
html = '<table><tr>'
print_col = 0
for i, (arch, row) in enumerate(cath_examples.iterrows()):
    cath_id = row[0]
    pdb_filename = os.path.join(pdb_dir, cath_id)

    if (i-sum(num_columns[:print_col])) % num_columns[print_col] == 0 and i > 0:
        print_col += 1
        html += '</tr><tr>'
    viewer_html = view_structure(pdb_filename, arch)._make_html()
    html += f'<td><div style="text-align: center;"><strong>{arch} {architecture_names[arch]}</strong></div>{viewer_html}</td>'
html += '</tr></table>'

# Display the grid
display(HTML(html))


"(1, 10) Mainly Alpha: Orthogonal Bundle You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: jupyter labextension install jupyterlab_3dmol","(1, 20) Mainly Alpha: Up-down Bundle You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: jupyter labextension install jupyterlab_3dmol"
"(2, 30) Mainly Beta: Roll You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: jupyter labextension install jupyterlab_3dmol","(2, 40) Mainly Beta: Beta Barrel You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: jupyter labextension install jupyterlab_3dmol"


## 3. Note the gaps

Since the protein domain structures are experimental determined, there are some regions of the domain that failed to resolve. These residue show up as gaps in the protein domain sequence and structure.

In [98]:
# Indices of the cath domain associated with the cath_id in the full protein
# that can be found using the pdb id in the PDB online.

example_cath_ids = ['3zq4C03', '3i9v600']
data[data['cath_id'].isin(example_cath_ids)]

,cath_id,class,architecture,topology,superfamily,resolution_in_angstroms,pdb_id,sequences,cath_indices
2,3zq4C03,3,10,20,580,3.0,3zq4,DIGNIVLRDRRILSEEGLVIVVVSIDMDDFKISAGPDLISRGFVIN...,"[(449, 555)]"
12,3i9v600,3,40,50,12280,3.1,3i9v,EREGILFTTLEKLVAWGRSNSLWPATFGLACCAIEMMASTDARQAD...,"[(15, 175)]"


In [102]:
# If you compare the indices that in the cath_indices range and not in the PDB file
# residue indices, for these examples you will get this

example_gaps = {'3zq4C03': [('C', 493, 501)],
                '3i9v600': [('6', 58, 74)]}

In [103]:
# We can visualize the gaps are red lines
import py3Dmol
from IPython.display import display, HTML

# Creating HTML table for the grid
html = '<table><tr>'
for cath_id, gap in example_gaps.items():
  viewer_html = view_structure(f'pdb_share/{cath_id}', cath_id, gaps=gap)._make_html()
  html += f'<td><div style="text-align: center;"><strong>{cath_id}</strong></div>{viewer_html}</td>'
html += '</tr></table>'

# Display the grid
display(HTML(html))

3zq4C03 You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: jupyter labextension install jupyterlab_3dmol,3i9v600 You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: jupyter labextension install jupyterlab_3dmol
